### Настройка среды

In [1]:
import os
import sys

sys.path.append(os.path.abspath(".."))

In [2]:
%pip install belashovplot

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
from belashovplot import TiledPlot
from utilities import *
from elements.modulators import Lens, PhaseModulator
from elements.propagators import FurrierPropagation
from elements.composition import CompositeModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from tqdm import tqdm
from math import sin, sqrt
import torch

### Цель эксперемента

Целью данного эксперемента является задание системы ...

### Настройка некогерентности

In [5]:
spatial_coherence = 50.0E-6
time_coherence = 10.0E-9
time_relaxation = 1.0E-6
mean_samples = 100

print(f"Временная когерентность:        {engineering(time_coherence, 'с')}")
print(f"Время релаксации:               {engineering(time_relaxation, 'c')}")
print(f"Пространственная когерентность: {engineering(spatial_coherence, 'м')}")

Временная когерентность:        10.0 нс
Время релаксации:               1.0 мкc
Пространственная когерентность: 50.0 мкм


## Система без линз

Описание системы

### Настройка параметров системы

In [15]:
length = 10.0E-3
wavelength = 500E-9
mask_unit_length = 10.0E-6
masks_amount = 5
cppp = 3

pixels = int(length/mask_unit_length) + 1
N = int(pixels*cppp)
distance = length * sqrt((mask_unit_length/wavelength)**2-1) * 0.5

print(f"Длинна волны:                             {engineering(wavelength, 'м')}")
print(f"Количество вычислительных пикселей:       {N}")
print(f"Количество пикселей маски:                {pixels}")
print(f"Количество модуляторов:                   {masks_amount}")
print(f"Размер оптических элементов:              {engineering(length, 'м')}")
print(f"Размер пикселя маски:                     {engineering(mask_unit_length, 'м')}")
print(f"Расстояние между слоями:                  {engineering(distance, 'м')}")

Длинна волны:                             500.0 нм
Количество вычислительных пикселей:       3000
Количество пикселей маски:                1000
Количество модуляторов:                   5
Размер оптических элементов:              10.0 мм
Размер пикселя маски:                     10.0 мкм
Расстояние между слоями:                  99.875 мм


### Инициализация оптической модели

In [17]:
propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.7)
modulators = [PhaseModulator(N, length, pixels) for i in range(masks_amount)]
elements = [modulators[0]]
for modulator in modulators[1:]:
    elements.append(propagation)
    elements.append(modulator)

incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, pixels, length)
chunker = CudaMemoryChunker()

model = CompositeModel(*elements)
model.wrap(chunker)
model.wrap(incoherent)

### Инициализация набора данных

In [ ]:
dataset = Dataset('Flowers', 100, N, N, torch.complex64)
dataset.train
dataset.test

## Система с одной линзой

Описание системы ...

### Настройка параметров системы

In [11]:
length = 20.0E-3
wavelength = 500E-9
mask_unit_length = 20.0E-6
masks_amount = 5
cppp = 3

pixels = int(length/mask_unit_length) + 1
N = int(pixels*cppp)
distance = length * sqrt((mask_unit_length/wavelength)**2-1)

focus = length / 2
b = distance*(masks_amount + 1)
H = length*focus / (b - focus)
a = b * H / length
h_pixels = int(pixels * H / length)

print(f"Длинна волны:                             {engineering(wavelength, 'м')}")
print(f"Количество вычислительных пикселей:       {N}")
print(f"Количество пикселей маски:                {pixels}")
print(f"Количество пикселей входного изображения: {h_pixels}")
print(f"Количество модуляторов:                   {masks_amount}")
print(f"Размер оптических элементов:              {engineering(length, 'м')}")
print(f"Размер пикселя маски:                     {engineering(mask_unit_length, 'м')}")
print(f"Размер входного изображения:              {engineering(H, 'м')}")
print(f"Расстояние между слоями:                  {engineering(distance, 'м')}")
print(f"Фокусное расстояние линзы:                {engineering(focus, 'м')}")
print(f"Расстояние до линзы:                      {engineering(a, 'м')}")
print(f"Расстояние после линзы:                   {engineering(b, 'м')}")

Длинна волны:                             500.0 нм
Количество вычислительных пикселей:       3000
Количество пикселей маски:                1000
Количество пикселей входного изображения: 2
Количество модуляторов:                   5
Размер оптических элементов:              20.0 мм
Размер пикселя маски:                     20.0 мкм
Размер входного изображения:              41.767 мкм
Расстояние между слоями:                  799.75 мм
Фокусное расстояние линзы:                10.0 мм
Расстояние до линзы:                      10.021 мм
Расстояние после линзы:                   4.798 м


### Инициализация модулей и оптической модели

In [7]:
р_pagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.7)
propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.7)
modulators = [PhaseModulator(N, length, pixels) for i in range(modulators_amount)]
elements = [modulators[0]]
for modulator in modulators[1:]:
    elements.append(propagation)
    elements.append(modulator)

incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, pixels, length)
chunker = CudaMemoryChunker()

model = CompositeModel(*elements)
model.wrap(chunker)
model.wrap(incoherent)

NameError: name 'modulators_amount' is not defined

### Выбор набора данных

In [ ]:
dataset = Dataset('Flowers', 100, N, N, torch.complex64)
dataset.train
dataset.test

### Тестирование модели